In [ ]:
include("../../src/CoherentStructures.jl") #this is terrible but neccessary during development (?)

In [ ]:
using StreamMacros, Main.CoherentStructures
using Tensors, Arpack, JLD2, OrdinaryDiffEq
using Plots
using OrdinaryDiffEq
const ODE = OrdinaryDiffEq

In [ ]:
JLD2.@load("Ocean_geostrophic_velocity.jld2");
UV = interpolateVF(Lon, Lat, Time, UT, VT);

In [ ]:
# Parameters
grid_resolution        = 100
quadrature_order       = 1
t_initial              = minimum(Time)
t_end                  = t_initial+90
scale                  = 1;  

In [ ]:
LL, UR = (-4.0, -34.0), (6.0, -28.0)
ctx, _ = regularTriangularGrid((grid_resolution, Integer(floor((UR[2] - LL[2])/(UR[1] - LL[1])*grid_resolution))), 
        LL, UR, quadrature_order = quadrature_order);

In [ ]:
function evaluate_on_grid(f,ctx)
    [f(node.x) for node in ctx.grid.nodes]
end

In [ ]:
function l1_error(reference,solution)
    sum(sum([abs.(a .- b) for (a,b) in zip(reference,solution)])) / (2*length(reference))
end

In [ ]:
function l2_error(reference,solution)
    sum(sqrt.(sum([(a .- b).^2 for (a,b) in zip(reference,solution)]))) / (2*length(reference))
end

In [ ]:
function linfty_error(reference,solution)
    sum(maximum([(a .- b) for (a,b) in zip(reference,solution)]))
end

In [ ]:
function T(solver, tolerance)
    return (x, param) -> flow(interp_rhs, x, [t_initial, t_end + scale*param], p=UV,
            tolerance=tolerance,solver=solver)[end]
end

In [ ]:
function Tdot_reference(solver, tolerance, param)
   return x -> UV(T(solver, tolerance)(x,param)...,t_end+param)*scale
end

In [ ]:
function Tdot_autodiff(solver, tolerance, param)
    return x -> Main.CoherentStructures.parameter_autodiff(T(solver, tolerance), x, param)
end

In [ ]:
function Tdot_finite_differences(solver, tolerance, param, diff)
    return x -> 
     (T(solver, tolerance)(x, param+diff) .- T(solver, tolerance)(x, param-diff)) ./ (2*diff)
end

In [ ]:
tolerance_range_BS5 = [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1e-10,1e-11,1e-12,1e-13,1e-14];

In [ ]:
reference_BS5 = [evaluate_on_grid(Tdot_reference(OrdinaryDiffEq.BS5(),tolerance,0),ctx) 
    for tolerance in tolerance_range_BS5];

In [ ]:
autodiff_BS5    = [evaluate_on_grid(Tdot_autodiff(OrdinaryDiffEq.BS5(),tolerance,0),ctx) 
    for tolerance in tolerance_range_BS5]
FD_1eminus2_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),tolerance,0,1e-2),ctx)
    for tolerance in tolerance_range_BS5]
FD_1eminus3_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),tolerance,0,1e-3),ctx)
    for tolerance in tolerance_range_BS5]
FD_1eminus4_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),tolerance,0,1e-4),ctx) 
    for tolerance in tolerance_range_BS5]
FD_1eminus5_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),tolerance,0,1e-5),ctx)
    for tolerance in tolerance_range_BS5];
FD_1eminus6_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),tolerance,0,1e-6),ctx)
    for tolerance in tolerance_range_BS5];
FD_1eminus7_BS5 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.BS5(),tolerance,0,1e-7),ctx)
    for tolerance in tolerance_range_BS5];

In [ ]:
l2_plot_BS5 = Plots.plot(tolerance_range_BS5,[l2_error(ref_tol,tol) 
        for (ref_tol,tol) in zip(reference_BS5,autodiff_BS5)], 
    xaxis=:log, yaxis=:log, marker=:dot, xflip=true, label="Autodiff BS5"
    ,fmt=:png,dpi=300,title="L2 error",legend=(0.1,0.25),legendfontsize=5)
Plots.plot!(tolerance_range_BS5,[l2_error(ref_tol,tol)
        for (ref_tol,tol) in zip(reference_BS5,FD_1eminus2_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-2, BS5")
Plots.plot!(tolerance_range_BS5,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_BS5,FD_1eminus3_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-3, BS5")
Plots.plot!(tolerance_range_BS5,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_BS5,FD_1eminus4_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-4, BS5")
Plots.plot!(tolerance_range_BS5,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_BS5,FD_1eminus5_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-5, BS5")
Plots.plot!(tolerance_range_BS5,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_BS5,FD_1eminus6_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-6, BS5")
Plots.plot!(tolerance_range_BS5,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_BS5,FD_1eminus7_BS5)], 
    marker=:dot, label="Finite Differences, δ=1e-7, BS5")
Plots.plot!(tolerance_range_BS5,[10x for x in tolerance_range_BS5], 
    linestyle=:dash,label="O(h)")

In [ ]:
tolerance_range_DP8 = [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1e-10,1e-11,1e-12,1e-13,1e-14];

In [ ]:
reference_DP8 = [evaluate_on_grid(Tdot_reference(OrdinaryDiffEq.DP8(),stepsize,0),ctx) 
    for stepsize in tolerance_range_DP8];

In [ ]:
autodiff_DP8     = [evaluate_on_grid(Tdot_autodiff(OrdinaryDiffEq.DP8(),tolerance,0),ctx) 
    for tolerance in tolerance_range_DP8]
FD_1eminus2_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),tolerance,0,1e-2),ctx)
    for tolerance in tolerance_range_DP8]
FD_1eminus3_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),tolerance,0,1e-3),ctx)
    for tolerance in tolerance_range_DP8]
FD_1eminus4_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),tolerance,0,1e-4),ctx) 
    for tolerance in tolerance_range_DP8]
FD_1eminus5_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),tolerance,0,1e-5),ctx)
    for tolerance in tolerance_range_DP8];
FD_1eminus6_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),tolerance,0,1e-6),ctx)
    for tolerance in tolerance_range_DP8];
FD_1eminus7_DP8 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.DP8(),tolerance,0,1e-7),ctx)
    for tolerance in tolerance_range_DP8];

In [ ]:
l2_plot_DP8 = Plots.plot(tolerance_range_DP8,[l2_error(ref_tol,tol) 
        for (ref_tol,tol) in zip(reference_DP8,autodiff_DP8)], 
    xaxis=:log, yaxis=:log, marker=:dot, xflip=true, label="Autodiff DP8"
    ,fmt=:png,dpi=300,title="L2 error",legend=(0.1,0.25),legendfontsize=5)
Plots.plot!(tolerance_range_DP8,[l2_error(ref_tol,tol) 
        for (ref_tol,tol) in zip(reference_DP8,FD_1eminus2_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-2, DP8")
Plots.plot!(tolerance_range_DP8,[l2_error(ref_tol,tol) 
         for (ref_tol,tol) in zip(reference_DP8,FD_1eminus3_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-3, DP8")
Plots.plot!(tolerance_range_DP8,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_DP8,FD_1eminus4_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-4, DP8")
Plots.plot!(tolerance_range_DP8,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_DP8,FD_1eminus5_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-5, DP8")
Plots.plot!(tolerance_range_DP8,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_DP8,FD_1eminus6_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-6, DP8")
Plots.plot!(tolerance_range_DP8,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_DP8,FD_1eminus7_DP8)], 
    marker=:dot, label="Finite Differences, δ=1e-7, DP8")
Plots.plot!(tolerance_range_DP8,[0.01x for x in tolerance_range_DP8], 
    linestyle=:dash,label="O(h)")

In [ ]:
tolerance_range_Vern7 = [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1e-10,1e-11,1e-12,1e-13,1e-14];

In [ ]:
reference_Vern7 = [evaluate_on_grid(Tdot_reference(OrdinaryDiffEq.Vern7(),tolerance,0),ctx) 
    for tolerance in tolerance_range_Vern7];

In [ ]:
autodiff_Vern7     = [evaluate_on_grid(Tdot_autodiff(OrdinaryDiffEq.Vern7(),tolerance,0),ctx) 
    for tolerance in tolerance_range_Vern7]
FD_1eminus2_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),tolerance,0,1e-2),ctx)
    for tolerance in tolerance_range_Vern7]
FD_1eminus3_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),tolerance,0,1e-3),ctx)
    for tolerance in tolerance_range_Vern7]
FD_1eminus4_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),tolerance,0,1e-4),ctx) 
    for tolerance in tolerance_range_Vern7]
FD_1eminus5_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),tolerance,0,1e-5),ctx)
    for tolerance in tolerance_range_Vern7];
FD_1eminus6_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),tolerance,0,1e-6),ctx)
    for tolerance in tolerance_range_Vern7];
FD_1eminus7_Vern7 = [evaluate_on_grid(Tdot_finite_differences(OrdinaryDiffEq.Vern7(),tolerance,0,1e-7),ctx)
    for tolerance in tolerance_range_Vern7];

In [ ]:
l2_plot_Vern7 = Plots.plot(tolerance_range_Vern7,[l2_error(ref_tol,tol) 
        for (ref_tol,tol) in zip(reference_Vern7,autodiff_Vern7)], 
    xaxis=:log, yaxis=:log, marker=:dot, xflip=true, label="Autodiff Vern7"
    ,fmt=:png,dpi=300,title="L2 error",legend=(0.1,0.25),legendfontsize=5)
Plots.plot!(tolerance_range_Vern7,[l2_error(ref_tol,tol) 
        for (ref_tol,tol) in zip(reference_Vern7,FD_1eminus2_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-2, Vern7")
Plots.plot!(tolerance_range_Vern7,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_Vern7,FD_1eminus3_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-3, Vern7")
Plots.plot!(tolerance_range_Vern7,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_Vern7,FD_1eminus4_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-4, Vern7")
Plots.plot!(tolerance_range_Vern7,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_Vern7,FD_1eminus5_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-5, Vern7")
Plots.plot!(tolerance_range_Vern7,[l2_error(ref_tol,tol) 
         for (ref_tol,tol) in zip(reference_Vern7,FD_1eminus6_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-6, Vern7")
Plots.plot!(tolerance_range_Vern7,[l2_error(ref_tol,tol)
         for (ref_tol,tol) in zip(reference_Vern7,FD_1eminus7_Vern7)], 
    marker=:dot, label="Finite Differences, δ=1e-7, Vern7")
Plots.plot!(tolerance_range_Vern7,[0.01x for x in tolerance_range_Vern7], 
    linestyle=:dash,label="O(h)")